In [ ]:
#read the csv text file
import pandas as pd
import numpy as np

percentage=0.01
nRows=3000001
nOfRowsToRead=int(nRows*percentage)

#read the csv text file
df = pd.read_csv('./Books_rating.csv', header=None , nrows=nOfRowsToRead)
df=df.to_numpy()

header= df[0]
data=df[1:]
reviews=data[:,-1]

print(reviews)
